In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#****Open Vino inference****

In [ ]:
# installing OpenVINO 

!wget https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2020
!touch /etc/apt/sources.list.d/intel-openvino-2020.list
!echo "deb https://apt.repos.intel.com/openvino/2020 all main" >> /etc/apt/sources.list.d/intel-openvino-2020.list

!apt update
!apt install intel-openvino-dev-ubuntu18-2020.4.287

!pip install test-generator==0.1.1
!source /opt/intel/openvino/bin/setupvars.sh

--2021-04-20 13:41:35--  https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
Resolving apt.repos.intel.com (apt.repos.intel.com)... 23.2.8.138, 2600:1408:5400:396::4b23, 2600:1408:5400:388::4b23
Connecting to apt.repos.intel.com (apt.repos.intel.com)|23.2.8.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 939 [binary/octet-stream]
Saving to: ‘GPG-PUB-KEY-INTEL-OPENVINO-2020’

GPG-PUB-KEY-INTEL-O 100%[===================>]     939  --.-KB/s    in 0s      

2021-04-20 13:41:35 (151 MB/s) - ‘GPG-PUB-KEY-INTEL-OPENVINO-2020’ saved [939/939]

OK
Get:1 https://apt.repos.intel.com/openvino/2020 all InRelease [5,659 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease 

In [ ]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import numpy as np

model = load_model('/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/model.h5')


In [ ]:
#convertinh .h5 model to frozen graphs

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
#path of the directory to saved model
frozen_out_path = '/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp'
# name of the .pb file
frozen_graph_filename = "frozen_graph"
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)
# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pb",
                  as_text=False)
# Save its text representation
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir=frozen_out_path,
                  name=f"{frozen_graph_filename}.pbtxt",
                  as_text=True)

------------------------------------------------------------
Frozen model layers: 
x
model/conv1/7x7_s2/Conv2D/ReadVariableOp/resource
model/conv1/7x7_s2/Conv2D/ReadVariableOp
model/conv1/7x7_s2/Conv2D
model/conv1/7x7_s2/bn/ReadVariableOp/resource
model/conv1/7x7_s2/bn/ReadVariableOp
model/conv1/7x7_s2/bn/ReadVariableOp_1/resource
model/conv1/7x7_s2/bn/ReadVariableOp_1
model/conv1/7x7_s2/bn/FusedBatchNormV3/ReadVariableOp/resource
model/conv1/7x7_s2/bn/FusedBatchNormV3/ReadVariableOp
model/conv1/7x7_s2/bn/FusedBatchNormV3/ReadVariableOp_1/resource
model/conv1/7x7_s2/bn/FusedBatchNormV3/ReadVariableOp_1
model/conv1/7x7_s2/bn/FusedBatchNormV3
model/activation/Relu
model/max_pooling2d/MaxPool
model/conv2_1_1x1_reduce/Conv2D/ReadVariableOp/resource
model/conv2_1_1x1_reduce/Conv2D/ReadVariableOp
model/conv2_1_1x1_reduce/Conv2D
model/conv2_1_1x1_reduce/bn/ReadVariableOp/resource
model/conv2_1_1x1_reduce/bn/ReadVariableOp
model/conv2_1_1x1_reduce/bn/ReadVariableOp_1/resource
model/conv2_1_1x1

'/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/frozen_graph.pbtxt'

In [ ]:

mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

pb_file = '/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/frozen_graph.pb'
output_dir = '/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp'
img_height = 224
input_shape = [1, img_height, img_height, 3]
input_shape_str = str(input_shape).replace(' ', '')

#Running Commands to run OPENVINO's Model Optimiser Converter 'mo_tf.py'
!source /opt/intel/openvino/bin/setupvars.sh
!python {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP16

[setupvars.sh] OpenVINO environment initialized
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/frozen_graph.pb
	- Path for generated IR: 	/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp
	- IR output name: 	frozen_graph
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,224,224,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementa

In [ ]:
!source /opt/intel/openvino/bin/setupvars.sh &&\
python /content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/inference.py

[setupvars.sh] OpenVINO environment initialized
2021-04-20 13:42:42.402492: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/inference.py:42: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin("CPU", plugin_dirs=plugin_dir)
/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/inference.py:44: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
/content/drive/MyDrive/Kaggle-Autism/models/h5/timestamp/inference.py:45: DeprecationWarning: 'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.
  assert len(net.inputs.keys()) == 1
len(test_data_gen)):
300
{'model/classi